In [1]:
import pandas as pd
from pandas import read_excel
from os import listdir

In [2]:
list_files = listdir("ods") #Nom des fichiers ods 

dico_files est un dictionnaire qui contient des dataframe

In [3]:
dico_files = {} #Clé = Nom du fichier, Valeur = dataframe
for file in list_files:
    dico_files[file] = read_excel(f"ods/{file}")
print(list_files)


ValueError: Worksheet index 0 is invalid, 0 worksheets found

Créer pour chaque dataframe une colonne election, année et tour dans le but de tous les merge à la fin tout en gardant la trace de quel résultat appartient à quelle élection/tour

In [5]:
i = 0
for key in dico_files.keys():
    dico_files[key]["election"] = list_files[i][0:3]
    dico_files[key]["année"] = list_files[i][3:5]
    dico_files[key]["tour"] = list_files[i][7]
    dico_files[key] = dico_files[key].drop(columns = 'Circonscription')
    i += 1

KeyError: "['Circonscription'] not found in axis"

Corriger des erreurs de typo présentes dans les données 

In [8]:
dico_files["mun20_T1.ods"] = dico_files["mun20_T1.ods"].rename(columns = {'Exprimes':'Exprimés'})
dico_files["mun20_T2.ods"] = dico_files["mun20_T1.ods"].rename(columns = {'Exprimes':'Exprimés'})
dico_files["leg24_T2.ods"] = dico_files["leg24_T2.ods"].rename(columns = {'Absentions':'Abstentions'})

In [11]:
dico_files.keys()

dict_keys(['leg22_T1.ods', 'leg22_T2.ods', 'leg24_T1.ods', 'leg24_T2.ods', 'mun20_T1.ods', 'mun20_T2.ods', 'pre22_T1.ods'])

### Valeurs manquantes

Il manquait des valeurs pour la ligne d'indice 134 que je suis allée chercher sur internet 

In [26]:
valeurs_de_remplacement = {'Inscrits': 1391, 'Abstentions': 878, "Votants":513, "Exprimés":501, "ED":25, "DROITE":154, "GAUCHE": 295, "EG":17, "DIVERS":10}  # Dictionnaire des valeurs à remplir
for col, valeur in valeurs_de_remplacement.items():
    dico_files["pre22_T1.ods"].loc[134, col] = valeur

### Merge tout mes dataframes 

### Traitements

Je vais merge tout les fichiers csv dans un seul et même dataframe 

In [28]:
df = pd.DataFrame()
for key in dico_files.keys():
    df = pd.concat([df ,dico_files[key]])

Je sais qu'il ne manque aucun quartiers de mairiedonc je retire toutes les lignes qui n'ont pas de quartiers de mairie

In [40]:
masque_nettoyage = df.loc[df["Quartiers Mairie"].isna() == True]

In [45]:
df = df.drop(index = masque_nettoyage.index)

J'ai encore des valeurs manquantes, et c'est normal, il y a généralement pas de partis d'extrème gauche ou divers en 2nd tour des élections. 

In [ ]:
df["Participation"] = df["Exprimés"]/df["Inscrits"]

In [50]:
df.to_csv("dataset2.csv")

In [52]:
df.columns

Index(['Bureau de vote', 'Quartier', 'Quartiers Mairie', 'Inscrits',
       'Abstentions', 'Votants', 'Exprimés', 'ED', 'DROITE', 'GAUCHE', 'EG',
       'DIVERS', 'election', 'année', 'tour'],
      dtype='object')